# Video Overlay Editor

This notebook allows you to overlay 4 images on a video at specific timestamps with scaling and rotation capabilities.

**Important:** Make sure your Jupyter kernel is set to use the `marathon` conda environment:
- Go to `Kernel` → `Change Kernel` → Select `marathon`
- Or install the kernel: `conda activate marathon && python -m ipykernel install --user --name=marathon`


In [48]:
# Setup: Install marathon kernel for Jupyter (run once if kernel not available)
# Uncomment and run this cell if you need to install the marathon kernel
# !conda activate marathon && python -m ipykernel install --user --name=marathon


In [49]:
# Verify conda environment and install required packages
# Make sure your Jupyter kernel is set to use the 'marathon' conda environment
import sys
print(f"Python executable: {sys.executable}")
print(f"Python version: {sys.version}")

# Check if we're in the marathon environment
if 'marathon' not in sys.executable:
    print(f"\n⚠️  WARNING: Not using marathon environment!")
    print(f"Current Python: {sys.executable}")
    print("Please select the 'marathon' kernel from: Kernel → Change Kernel → marathon")
else:
    print(f"\n✓ Using marathon environment: {sys.executable}")

# Install/upgrade required packages in the current environment
print("\nInstalling/updating packages...")
%pip install moviepy pillow numpy


Python executable: /opt/anaconda3/envs/marathon/bin/python
Python version: 3.13.9 | packaged by Anaconda, Inc. | (main, Oct 21 2025, 19:11:29) [Clang 20.1.8 ]

✓ Using marathon environment: /opt/anaconda3/envs/marathon/bin/python

Installing/updating packages...
Note: you may need to restart the kernel to use updated packages.


In [50]:
# Verify we're in the marathon environment and import required modules
import sys
import os

# Check if we're in the marathon environment
if 'marathon' not in sys.executable:
    print(f"WARNING: Not using marathon environment!")
    print(f"Current Python: {sys.executable}")
    print("Please select the 'marathon' kernel from the kernel menu in Jupyter")
else:
    print(f"✓ Using marathon environment: {sys.executable}")

# Import required modules
# Note: moviepy 2.x uses direct imports (not moviepy.editor)
try:
    from moviepy import VideoFileClip, ImageClip, CompositeVideoClip
    from PIL import Image
    import numpy as np
    from pathlib import Path
    print("✓ All modules imported successfully")
except ImportError as e:
    print(f"✗ Import error: {e}")
    print("Please run the installation cell above first, then restart the kernel")


✓ Using marathon environment: /opt/anaconda3/envs/marathon/bin/python
✓ All modules imported successfully


## Configuration

Define your video path, image paths, timestamps, positions, scales, and rotations for each overlay.


In [ ]:
# Video input path
VIDEO_PATH = "/Users/phoenixa/Documents/projects/marathon/sample.mp4"

# Output video path
OUTPUT_PATH = "/Users/phoenixa/Documents/projects/marathon/output_video.mp4"

# Overlay configuration for 4 images
# Each overlay has:
#   - image_path: path to the image file
#   - start_time: when to start showing the image (in seconds)
#   - duration: how long to show the image (in seconds)
#   - position: (x, y) position on the video (can be 'center', (x, y) tuple, or function)
#   - scale: scaling factor (1.0 = original size, 0.5 = half size, 2.0 = double size)
#   - rotation: rotation angle in degrees (positive = clockwise)
#   - opacity: opacity from 0.0 (transparent) to 1.0 (opaque)

OVERLAYS = [
     {
        "image_path": "WHITE_FRAME",  # Special value, will generate a white image at runtime
        "start_time": 21.0,  # Set the appropriate start time
        "duration": 5.0,  # Show for 5 seconds
        "position": "center",  # Full video overlay, position is centered
        "width": 1.0,   # 100% of video width
        "height": 1.0,  # 100% of video height
        "scale": 1.0,   # Redundant, included for compatibility
        "rotation": 0,
        "opacity": 1.0  # Fully opaque
    },
    {
        "image_path": "/Users/phoenixa/Documents/projects/marathon/Edited/Sarthi Studios-2.jpg",  # Replace with your image path
        "start_time": 21.0,  # Start at 21 seconds
        "duration": 5.0,  # Show for 5 seconds
        "position": "center",  # Position at (100, 100) from top-left
        "scale": 0.2,  # Half size
        "rotation": 0,  # No rotation
        "opacity": 1.0  # Fully opaque
    },
   
    {
        "image_path": "/Users/phoenixa/Documents/projects/marathon/Edited/Sarthi Studios-4.jpg",  # Replace with your image path
        "start_time": 22.0,  # Start at 22 seconds
        "duration": 4.0,  # Show for 4 seconds
        "position": "center",  # Center of the video
        # Set EITHER 'scale' or both 'width' and/or 'height' in pixels or as a fraction of video dimensions. If width/height are set, they override 'scale'.
        "scale": 0.2,  # 10% of original size -- ignored if width or height below are set
        # Example: "width": 0.2 means 20% of video width, or use absolute pixels, eg. "width": 400
        # Uncomment one or both below to set image size relative to video dimensions
        # "width": 0.5,  # 20% of video width
        # "height": 0.15, # 15% of video height
        "rotation": 10,  # Rotate 10 degrees
        "opacity": 1  # Fully opaque
    },
    {
        "image_path": "/Users/phoenixa/Documents/projects/marathon/Edited/Sarthi Studios-5.jpg",  # Replace with your image path
        "start_time": 23.0,  # Start at 23 seconds
        "duration": 3.0,  # Show for 3 seconds
        "position": "center",  # Position at (500, 300)
        "scale": 0.2,  # 70% of original size
        "rotation": -10,  # Rotate -30 degrees (counter-clockwise)
        "opacity": 1.0
    },
    {
        "image_path": "/Users/phoenixa/Documents/projects/marathon/Edited/Sarthi Studios-6.jpg",  # Replace with your image path
        "start_time": 24.0,  # Start at 24 seconds
        "duration": 2.0,  # Show for 2 seconds
        "position": "center",  # Position at (50, 50)
        "scale": 0.2,  # 40% of original size
        "rotation": 20,  # Rotate 90 degrees
        "opacity": 1
    }
]


## Helper Functions

Functions to handle image transformations (scale, rotation) and positioning.


In [52]:
def transform_image(image_path, scale=1.0, rotation=0, opacity=1.0):
    """
    Load and transform an image with scaling, rotation, and opacity.
    
    Args:
        image_path: Path to the image file
        scale: Scaling factor (1.0 = original size)
        rotation: Rotation angle in degrees
        opacity: Opacity from 0.0 to 1.0
    
    Returns:
        PIL Image object with transformations applied
    """
    # Load image
    img = Image.open(image_path).convert("RGBA")
    
    # Apply scaling
    if scale != 1.0:
        new_size = (int(img.width * scale), int(img.height * scale))
        img = img.resize(new_size, Image.Resampling.LANCZOS)
    
    # Apply rotation
    if rotation != 0:
        img = img.rotate(-rotation, expand=True, fillcolor=(0, 0, 0, 0))
    
    # Apply opacity
    if opacity < 1.0:
        alpha = img.split()[3]
        alpha = alpha.point(lambda p: int(p * opacity))
        img.putalpha(alpha)
    
    return img


def get_position(position, video_size, image_size):
    """
    Calculate the actual position coordinates.
    
    Args:
        position: Can be 'center', (x, y) tuple, or a function
        video_size: (width, height) of the video
        image_size: (width, height) of the image
    
    Returns:
        (x, y) tuple for the position
    """
    if position == "center":
        return ("center", "center")
    elif callable(position):
        return position(video_size, image_size)
    else:
        return position


## Main Processing Function

This function processes the video and applies all overlays.


In [ ]:
def overlay_images_on_video(video_path, overlays, output_path):
    """
    Overlay multiple images on a video at specific timestamps with transformations.
    
    Args:
        video_path: Path to input video
        overlays: List of overlay configurations
        output_path: Path to save output video
    """
    # Load the video
    print(f"Loading video: {video_path}")
    video = VideoFileClip(video_path)
    video_duration = video.duration
    video_size = (video.w, video.h)
    
    print(f"Video loaded: {video_size[0]}x{video_size[1]}, duration: {video_duration:.2f}s")
    
    # Prepare overlay clips
    overlay_clips = []
    
    for i, overlay_config in enumerate(overlays):
        image_path = overlay_config["image_path"]
        
        start_time = overlay_config["start_time"]
        duration = overlay_config["duration"]
        end_time = start_time + duration

        scale = overlay_config.get("scale", 1.0)
        rotation = overlay_config.get("rotation", 0)
        opacity = overlay_config.get("opacity", 1.0)
        position = overlay_config.get("position", "center")
        width = overlay_config.get("width", None)
        height = overlay_config.get("height", None)
        
        # Check if overlay is within video duration
        if start_time >= video_duration:
            print(f"Warning: Overlay {i+1} starts after video ends, skipping...")
            continue
        
        # Handle special "WHITE_FRAME" case
        if image_path == "WHITE_FRAME":
            print(f"\nProcessing overlay {i+1}:")
            print(f"  Image: WHITE_FRAME (generating white image)")
            print(f"  Time: {start_time:.2f}s - {end_time:.2f}s ({duration:.2f}s)")
            
            # Calculate dimensions
            if width is not None:
                if width <= 1.0:
                    img_width = int(video_size[0] * width)
                else:
                    img_width = int(width)
            else:
                img_width = video_size[0]
                
            if height is not None:
                if height <= 1.0:
                    img_height = int(video_size[1] * height)
                else:
                    img_height = int(height)
            else:
                img_height = video_size[1]
            
            # Create white image with RGBA (white with full opacity)
            white_img = Image.new("RGBA", (img_width, img_height), (255, 255, 255, int(255 * opacity)))
            
            # Apply rotation if needed
            if rotation != 0:
                white_img = white_img.rotate(-rotation, expand=True, fillcolor=(0, 0, 0, 0))
            
            # Convert PIL image to numpy array
            img_array = np.array(white_img)
            
        else:
            # Check if image exists
            if not os.path.exists(image_path):
                print(f"Warning: Image {i+1} not found at {image_path}, skipping...")
                continue
            
            print(f"\nProcessing overlay {i+1}:")
            print(f"  Image: {image_path}")
            print(f"  Time: {start_time:.2f}s - {end_time:.2f}s ({duration:.2f}s)")
            print(f"  Scale: {scale}, Rotation: {rotation}°, Opacity: {opacity}")
            
            # Transform the image
            transformed_img = transform_image(image_path, scale=scale, rotation=rotation, opacity=opacity)
            
            # Apply width/height if specified (override scale)
            if width is not None or height is not None:
                current_w, current_h = transformed_img.size
                
                if width is not None:
                    if width <= 1.0:
                        new_width = int(video_size[0] * width)
                    else:
                        new_width = int(width)
                else:
                    new_width = current_w
                    
                if height is not None:
                    if height <= 1.0:
                        new_height = int(video_size[1] * height)
                    else:
                        new_height = int(height)
                else:
                    new_height = current_h
                
                transformed_img = transformed_img.resize((new_width, new_height), Image.Resampling.LANCZOS)
            
            # Convert PIL image to numpy array
            img_array = np.array(transformed_img)
        
        # Create ImageClip from the transformed image
        # In moviepy 2.x, duration is set in constructor, and use with_* methods instead of set_*
        img_clip = ImageClip(img_array, duration=duration)
        img_clip = img_clip.with_start(start_time)
        
        # Set position
        pos = get_position(position, video_size, (img_array.shape[1], img_array.shape[0]))
        img_clip = img_clip.with_position(pos)
        
        overlay_clips.append(img_clip)
        print(f"  ✓ Overlay {i+1} prepared")
    
    # Composite all clips
    print(f"\nCompositing {len(overlay_clips)} overlays on video...")
    final_video = CompositeVideoClip([video] + overlay_clips)
    
    # Write the output video
    print(f"\nWriting output video to: {output_path}")
    final_video.write_videofile(
        output_path,
        codec='libx264',
        audio_codec='aac',
        fps=video.fps,
        preset='medium',
        threads=4
    )
    
    # Clean up
    video.close()
    final_video.close()
    
    print(f"\n✓ Video processing complete! Output saved to: {output_path}")


## Execute Video Processing

**Important:** If you get errors about `set_duration` or `set_start`, make sure you've re-run the function definition cell above (the cell with `def overlay_images_on_video`) to load the updated code with moviepy 2.x API.

Run this cell to process the video with all overlays.


In [54]:
# Process the video
overlay_images_on_video(VIDEO_PATH, OVERLAYS, OUTPUT_PATH)


Loading video: /Users/phoenixa/Documents/projects/marathon/sample.mp4
Video loaded: 1080x1920, duration: 37.70s

Compositing 0 overlays on video...

Writing output video to: /Users/phoenixa/Documents/projects/marathon/output_video.mp4
MoviePy - Building video /Users/phoenixa/Documents/projects/marathon/output_video.mp4.
MoviePy - Writing audio in output_videoTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
MoviePy - Writing video /Users/phoenixa/Documents/projects/marathon/output_video.mp4



MoviePy - Done !
MoviePy - video ready /Users/phoenixa/Documents/projects/marathon/output_video.mp4

✓ Video processing complete! Output saved to: /Users/phoenixa/Documents/projects/marathon/output_video.mp4


## Advanced: Dynamic Positioning

You can also use functions for dynamic positioning (e.g., moving images across the screen).


In [55]:
# Example: Function to position image at bottom-right
def bottom_right(video_size, image_size):
    """Position image at bottom-right corner."""
    return (video_size[0] - image_size[0], video_size[1] - image_size[1])

# Example: Function to position image at top-center
def top_center(video_size, image_size):
    """Position image at top-center."""
    return ("center", 0)

# Example usage in OVERLAYS:
# "position": bottom_right  # Use function for dynamic positioning


## Preview Video Info

Check video properties before processing.


In [56]:
# Preview video information
def preview_video_info(video_path):
    """Display video information."""
    video = VideoFileClip(video_path)
    print(f"Video: {video_path}")
    print(f"Duration: {video.duration:.2f} seconds")
    print(f"Size: {video.w}x{video.h} pixels")
    print(f"FPS: {video.fps}")
    print(f"Codec: {video.codec}")
    video.close()

# Uncomment to preview:
# preview_video_info(VIDEO_PATH)
